In [ ]:
using DrWatson
@quickactivate "masterarbeit"

In [ ]:
using Random
using LaTeXStrings
using GLMakie
using StatsBase

In [ ]:
Makie.inline!(true)
fontsize_theme = Theme(fontsize=35)
set_theme!(fontsize_theme)
wblue = Makie.wong_colors()[1]
worange = Makie.wong_colors()[2]
wgreen = Makie.wong_colors()[3]
wpink = Makie.wong_colors()[4]
wlblue = Makie.wong_colors()[5]
worange = Makie.wong_colors()[6]
wyellow = Makie.wong_colors()[7];

In [ ]:
using Revise

In [ ]:
using masterarbeit

In [ ]:
include(joinpath(projectdir(),"src/vegas.jl"))

In [ ]:
omega = 5.12
dphi = 10.0
f = x -> comptonf(x, omega, dphi)[1]
vegasstart = [-1.0, 0.0]
vegasend = [1.0, 1.0]
xticks = [-1.0, -0.5, 0.0, 0.5, 1.0]
yticks = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
xs = LinRange(-1.0, 1.0, 1001)
ys = LinRange(0.0, 1.0, 1001)
fevals = sample -> f([sample[1], sample[2]]);

In [ ]:
res = vegas(f, vegasstart, vegasend)
weights = res.weights;

In [ ]:
samples, Js = sample_from_adaptive_grid(res, 10^7)
weights = Js .* fevals.(samples);

In [ ]:
sample_from_adaptive_grid(res, 10^7)

In [ ]:
sx = [s[1] for s in samples]
sy = [s[2] for s in samples]
histo = fit(Histogram, (sx, sy), nbins=100)
histo_n = StatsBase.normalize(histo, mode=:pdf)
fig = Figure(size=(1200,1000), figure_padding=40)
ax = Axis(fig[1,1], xlabel=L"\cos{\theta}", ylabel=L"\overline{\omega'}", xticks=xticks, yticks=yticks, aspect=1, xlabelsize=50, ylabelsize=50)
hm = heatmap!(histo.edges[1], histo.edges[2], histo_n.weights, colorrange=(0,25))
fig[1, 2] = GridLayout(width = 20)
Colorbar(fig[1,3], hm, width=40)
#save("compton_vegas_samples_0.1_truecolor.png", fig)
fig

In [ ]:
mcint = res.integral_estimate
mcerror = res.standard_deviation
println("mc integral = $mcint")
println("standard deviation = $mcerror")

In [ ]:
wi = weights
wi_n = weights ./ mcint
w_avg = mean(wi_n)
w_max = maximum(wi_n)
uweff = w_avg / w_max
println("mean weight = $w_avg")
println("max weight = $w_max")
println("unweighting efficiency = $uweff")

In [ ]:
function vegaswithgrid()
    fig = Figure(size=(1200,1000), figure_padding=40)
    ax2 = Axis(fig[1,1], xticks=xticks, yticks=yticks,
        aspect=1, xlabel=L"\cos{\theta}", ylabel=L"\overline{\omega'}", xlabelsize=50, ylabelsize=50)
    zs = [f([x,y])[1] for x in xs, y in ys]
    zs_n = zs ./ mcint
    hm = heatmap!(xs, ys, zs_n, colorrange=(0,25))
    vlines!(res.adaptive_grid[:,1][1:50:end], color=:white, linewidth=2)
    hlines!(res.adaptive_grid[:,2][1:50:end], color=:white, linewidth=2)
    fig[1, 2] = GridLayout(width = 20)
    Colorbar(fig[1,3], hm, width=40)
    #save("compton_vegas_grid_0.1.png", fig)
    return fig
end 
fig = vegaswithgrid()
fig

In [ ]:
using JLD2

In [ ]:
#save_object("vegas_weights_0.1.jld2", wi_n)